In [ ]:
from pathlib import Path
from ra2ce.analysis.analysis_config_wrapper import AnalysisConfigWrapper
from ra2ce.analysis.analysis_input_wrapper import AnalysisInputWrapper
from ra2ce.ra2ce_handler import Ra2ceHandler
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.analysis.analysis_config_data.analysis_config_data import (
    AnalysisConfigData,
    AnalysisSectionAdaptation,
    AnalysisSectionAdaptationOption,
    AnalysisSectionDamages,
    AnalysisSectionLosses,
)
from ra2ce.analysis.analysis_config_data.enums.analysis_damages_enum import (
    AnalysisDamagesEnum,
)
from ra2ce.analysis.analysis_config_data.enums.analysis_enum import AnalysisEnum
from ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum import (
    AnalysisLossesEnum,
)
from ra2ce.analysis.analysis_config_data.enums.damage_curve_enum import DamageCurveEnum
from ra2ce.analysis.analysis_config_data.enums.event_type_enum import EventTypeEnum
from ra2ce.analysis.analysis_config_data.enums.traffic_period_enum import (
    TrafficPeriodEnum,
)
from ra2ce.analysis.analysis_config_data.enums.trip_purpose_enum import TripPurposeEnum
from ra2ce.analysis.analysis_config_data.enums.weighing_enum import WeighingEnum
from ra2ce.analysis.analysis_config_wrapper import AnalysisConfigWrapper
from ra2ce.analysis.analysis_input_wrapper import AnalysisInputWrapper
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.network_config_data import (
    HazardSection,
    NetworkConfigData,
    NetworkSection,
)
from ra2ce.network.network_config_wrapper import NetworkConfigWrapper
from ra2ce.analysis.adaptation.adaptation import Adaptation
from ra2ce.analysis.analysis_config_wrapper import AnalysisConfigWrapper
from ra2ce.analysis.analysis_input_wrapper import AnalysisInputWrapper


c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

root_dir = Path("data", "adaptation")
static_path = root_dir.joinpath("static")
hazard_path =static_path.joinpath("hazard")
network_path = static_path.joinpath("network")
output_path=root_dir.joinpath("output")
tif_files = list(hazard_path.glob("*.tif"))
_input_path = root_dir.joinpath("input")

In [3]:

_network_section = NetworkSection(
    source= SourceEnum.SHAPEFILE,       
    primary_file = [network_path.joinpath("network.shp")], 
    file_id = "ID",
    link_type_column='highway',
    save_gpkg=True
)

_hazard = HazardSection(
    hazard_map=[Path(file) for file in hazard_path.glob("*.tif")],
    hazard_field_name= ['waterdepth'],
    aggregate_wl = AggregateWlEnum.MEAN,
    hazard_crs = 'EPSG:4326' 
)


_network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=static_path,
    output_path=output_path,
    network=_network_section,
    hazard=_hazard
    )

In [4]:

handler = Ra2ceHandler.from_config(_network_config_data, None)

handler.configure()
handler.run_analysis()

In [ ]:
def get_losses_section(analysis: AnalysisLossesEnum) -> AnalysisSectionLosses:
    return AnalysisSectionLosses(
            analysis=analysis,
            event_type=EventTypeEnum.EVENT,
            weighing=WeighingEnum.TIME,
            threshold=0,
            production_loss_per_capita_per_hour=42,
            hours_per_traffic_period=8,
            traffic_period=TrafficPeriodEnum.DAY,
            trip_purposes=[
                TripPurposeEnum.BUSINESS,
                TripPurposeEnum.COMMUTE,
                TripPurposeEnum.FREIGHT,
                TripPurposeEnum.OTHER,
            ],
            resilience_curves_file=_input_path.joinpath("resilience_curve.csv"),
            traffic_intensities_file=_input_path.joinpath("traffic_intensities.csv"),
            values_of_time_file=_input_path.joinpath("values_of_time.csv"),
            save_gpkg=True,
            save_csv=True,
        )

_damages_section = AnalysisSectionDamages(
        analysis=AnalysisDamagesEnum.DAMAGES,
        event_type=EventTypeEnum.EVENT,
        damage_curve=DamageCurveEnum.MAN,
        save_gpkg=True,
        save_csv=True,
    )

    # - losses
_multi_link_losses_section = get_losses_section(
        AnalysisLossesEnum.MULTI_LINK_LOSSES
    )


# - adaptation
_adaptation_options = [
    AnalysisSectionAdaptationOption(
            id="AO0",
            name="No adaptation",
        ),
        AnalysisSectionAdaptationOption(
            id="AO1",
            name="Cheap construction, expensive maintenance",
            construction_cost=1000.0,
            construction_interval=10.0,
            maintenance_cost=200.0,
            maintenance_interval=3.0,
        ),
        AnalysisSectionAdaptationOption(
            id="AO2",
            name="Expensive construction, cheap maintenance",
            construction_cost=5000.0,
            construction_interval=100.0,
            maintenance_cost=50.0,
            maintenance_interval=3.0,
        ),
]
_adaptation_section = AnalysisSectionAdaptation(
        analysis=AnalysisEnum.ADAPTATION,
        losses_analysis=AnalysisLossesEnum.MULTI_LINK_LOSSES,
        adaptation_options=_adaptation_options,
        discount_rate=0.025,
        time_horizon=20,
    )

_analysis_data = AnalysisConfigData(
        root_path=root_dir,
        input_path=_input_path,
        static_path=static_path,
        output_path=output_path,
        analyses=[
            _damages_section,
            _multi_link_losses_section,
            _adaptation_section,
        ],
        aggregate_wl=AggregateWlEnum.MEAN,
    )


In [ ]:

_network_config = NetworkConfigWrapper.from_data(None, _network_config_data)
valid_analysis_ini = root_dir.joinpath("analyses.ini")


_analysis_config = AnalysisConfigWrapper.from_data_with_network(
        valid_analysis_ini, _analysis_data, _network_config
    )

_analysis_input = AnalysisInputWrapper.from_input(
        analysis=_analysis_config.config_data.adaptation,
        analysis_config=_analysis_config,
        graph_file=_analysis_config.graph_files.base_network,
        graph_file_hazard=_analysis_config.graph_files.base_network_hazard,
    )



valid_adaptation_config = (_analysis_input, _analysis_config)
_adaptation = Adaptation(valid_adaptation_config[0], valid_adaptation_config[1])

# 2. Run test.
_result = _adaptation.run_benefit()
print(_result)


# Add modification of climate factor, discount_rate, time_horizon, .... 

2024-12-04 10:51:16 PM - [damage_network_base.py:114] - root - WARNING - Of the 235 road segments, only 26 had lane data, so for 209 the '
                                    lane data will be interpolated from the existing data
2024-12-04 10:51:16 PM - [damage_network_base.py:127] - root - WARNING - Interpolated the missing lane data as follows: {'secondary': 2.0, 'tertiary': 2.0, 'trunk': 2.0}
c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
2024-12-04 10:51:17 PM - [damage_network_base.py:114] - root - WARNING - Of the 235 road segments, only 26 had lane data, so for 209 the '
                                    lane data wil

     link_id   ID     highway  avgspeed  \
0          1    1    tertiary      58.0   
1         45   45    tertiary      60.0   
2         12   12    tertiary      58.0   
3        120  120    tertiary      58.0   
4         11   11    tertiary      58.0   
..       ...  ...         ...       ...   
230      253  253       trunk      50.0   
231      246  246       trunk      50.0   
232      249  249  trunk_link      50.0   
233      248  248       trunk      50.0   
234      257  257       trunk      50.0   

                                              geometry lanes  length maxspeed  \
0    LINESTRING (34.87673 -19.85047, 34.87642 -19.8...   nan    70.0      nan   
1    LINESTRING (34.88155 -19.84906, 34.88017 -19.8...   nan   529.0       60   
2    LINESTRING (34.87606 -19.85055, 34.87628 -19.8...   nan    71.0      nan   
3    LINESTRING (34.87614 -19.85074, 34.87611 -19.8...   nan    36.0      nan   
4    LINESTRING (34.87606 -19.85055, 34.87610 -19.8...   nan    24.0      nan 

c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [11]:
_result.head(5)

,link_id,ID,highway,avgspeed,geometry,lanes,length,maxspeed,bridge,node_A,...,time,AO0_dam_EV1_al,AO0_vlh_EV1_me_total,AO0_impact,AO1_dam_EV1_al,AO1_vlh_EV1_me_total,AO1_impact,AO2_dam_EV1_al,AO2_vlh_EV1_me_total,AO2_impact
0,1,1,tertiary,58.0,"LINESTRING (34.87673 -19.85047, 34.87642 -19.8...",nan,70.0,nan,nan,0,...,0.001,NaN,0.0000,0.0000,NaN,0.0000,0.0000,NaN,0.0000,0.0000
1,45,45,tertiary,60.0,"LINESTRING (34.88155 -19.84906, 34.88017 -19.8...",nan,529.0,60,nan,0,...,0.009,NaN,1.3485,1.3485,NaN,1.3485,1.3485,NaN,1.3485,1.3485
2,12,12,tertiary,58.0,"LINESTRING (34.87606 -19.85055, 34.87628 -19.8...",nan,71.0,nan,nan,0,...,0.001,NaN,0.0000,0.0000,NaN,0.0000,0.0000,NaN,0.0000,0.0000
3,120,120,tertiary,58.0,"LINESTRING (34.87614 -19.85074, 34.87611 -19.8...",nan,36.0,nan,nan,1,...,0.001,NaN,0.0000,0.0000,NaN,0.0000,0.0000,NaN,0.0000,0.0000
4,11,11,tertiary,58.0,"LINESTRING (34.87606 -19.85055, 34.87610 -19.8...",nan,24.0,nan,nan,1,...,0.000,NaN,0.0000,0.0000,NaN,0.0000,0.0000,NaN,0.0000,0.0000
